# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import os
import sys
import random
from scipy.stats import linregress
from citipy import citipy
import json

module_path = os.path.abspath(os.path.join('..'))
#Include api directory in path
if module_path not in sys.path:
    sys.path.append(module_path + "\\api")

# Import API key
from api_keys import weather_api_key

weather_key = weather_api_key

weather_base_url = "http://api.openweathermap.org/data/2.5/weather?"


# Output File (CSV)

output_data_file = "../output_data/cities.csv"





print(weather_api_key)

39b4c8c3c2107953741b12b2cc76dd00


## Genereate Random Cities

In [3]:
# Incorporated citipy to determine city based on latitude and longitude
#from citipy import citipy

#Create function for resuse in case ever wanting to create sample cities in future.
def gen_rand_cities(dict):
    coord_list = []
    min_lat = dict["lat_min"]
    max_lat = dict["lat_max"]
    min_long = dict["long_min"]
    max_long = dict["long_max"]
    samp_size = dict["samp_size"]
    x = 0
    city = []
    country = []
    while x < samp_size:
 
        lat_rand = min_lat + (max_lat - min_lat) * random.random()
        long_rand = min_long + (max_long - min_long) * random.random() 
        city_obj = citipy.nearest_city(lat_rand, long_rand) 
        #Don't duplicate city names
        if city_obj.city_name not in city:                  
            city.append(city_obj.city_name)
            country.append(city_obj.country_code)
            #print(f"{x} {city_obj.city_name} {city_obj.country_code}")
            x = x + 1
            
              
    funct_df = pd.DataFrame ( {
                              "city": city,
                              "country":country,             
                             })
    return(funct_df)

#Set dictionary to pass into function that will generate city list by 
#lattitude/longitude ranges
city_geo_params = {"lat_min":-90,
                   "lat_max":90,
                   "long_min":-180,
                   "long_max":180,
                   "samp_size":500};



city_df = gen_rand_cities(city_geo_params)     

len(city_df)







{
    "coord": {
        "lon": -159.319,
        "lat": 22.0752
    },
    "weather": [
        {
            "id": 800,
            "main": "Clear",
            "description": "clear sky",
            "icon": "01d"
        }
    ],
    "base": "stations",
    "main": {
        "temp": 298.26,
        "feels_like": 297.06,
        "temp_min": 297.59,
        "temp_max": 299.15,
        "pressure": 1018,
        "humidity": 61
    },
    "visibility": 10000,
    "wind": {
        "speed": 5.14,
        "deg": 70
    },
    "clouds": {
        "all": 1
    },
    "dt": 1612132267,
    "sys": {
        "type": 1,
        "id": 7873,
        "country": "US",
        "sunrise": 1612113348,
        "sunset": 1612153540
    },
    "timezone": -36000,
    "id": 5848280,
    "name": "Kapaa",
    "cod": 200
}
{'coord': {'lon': -159.319, 'lat': 22.0752}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 298.26, 'feels_like'

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [26]:
#Initialize cities that will create data frame
cities = []
countries = []
lattitudes = []
longitudes = []
max_temps = []
humidities = []
cloudiness = []
windspeeds = []
countries = []
dates = []


#User itterrows to loop through data frame
for index, row in city_df.iterrows():
    city = row['city']
    country = row['country'].upper()
    params = {"appid": weather_key, "q": city + ',' + country, "units":"imperial"}
    #Exception block to handle when city can't be found.
    try:
        response = requests.get(weather_base_url,params=params).json()
        lattitudes.append(response["coord"]["lat"])
        longitudes.append(response["coord"]["lon"])
        max_temps.append(response["main"]["temp_max"])
        humidities.append(response["main"]["humidity"])
        cloudiness.append(response["clouds"]["all"])
        windspeeds.append(response["wind"]["speed"])
        countries.append(country)
        cities.append(city)
        dates.append(response["dt"])
        print(f'{index}- {city} ({response["coord"]["lat"]}:{response["coord"]["lon"]})')                              
    except:
        print(f'{city} ({country}) Not Found')





0- ancud (-41.8697:-73.8203)
1- erenhot (43.6475:111.9767)
2- misratah (32.3754:15.0925)
taolanaro (MG) Not Found
4- villamartin (36.8598:-5.6449)
5- vikhorevka (56.1201:101.1636)
6- bluff (-46.6:168.3333)
7- rikitea (-23.1203:-134.9692)
8- yellowknife (62.456:-114.3525)
9- san cristobal (-0.9045:-89.6174)
10- salina (38.8403:-97.6114)
11- zhigansk (66.7697:123.3711)
12- vila franca do campo (37.7167:-25.4333)
13- andenes (69.3143:16.1194)
14- tiznit (29.5833:-9.5)
15- jamestown (-15.9387:-5.7168)
16- ushuaia (-54.8:-68.3)
17- lorengau (-2.0226:147.2712)
18- broome (-17.9667:122.2333)
19- upata (8.0086:-62.3989)
20- puerto ayora (-0.7393:-90.3518)
21- teguise (29.0605:-13.564)
faya (TD) Not Found
23- hermanus (-34.4187:19.2345)
24- vaini (-21.2:-175.2)
25- levelland (33.5873:-102.378)
26- pandan (14.0453:124.1694)
bababe (MR) Not Found
28- hasaki (35.7333:140.8333)
29- cape town (-33.9258:18.4232)
mrirt (MA) Not Found
31- port alfred (-33.5906:26.891)
32- east london (-33.0153:27.9116)

259- itarema (-2.9248:-39.9167)
260- kathu (7.9178:98.3332)
sentyabrskiy (RU) Not Found
262- kilindoni (-7.9139:39.6668)
maarianhamina (FI) Not Found
264- ushtobe (45.2514:77.9838)
265- igarka (67.4667:86.5833)
266- olesno (50.877:18.4209)
267- angangueo (19.6194:-100.2853)
268- los llanos de aridane (28.6585:-17.9182)
269- taltal (-25.4:-70.4833)
270- sorland (67.667:12.6934)
tome-acu (BR) Not Found
272- piacabucu (-10.4056:-36.4344)
273- hervey bay (-25.2986:152.8535)
274- vulcan (50.4001:-113.2519)
275- yazykovo (55.2199:56.1353)
276- sioux lookout (50.1001:-91.917)
277- talnakh (69.4865:88.3972)
278- vila praia de ancora (41.811:-8.8526)
279- pitimbu (-7.4706:-34.8086)
280- mangan (27.5167:88.5333)
281- cabo san lucas (22.8909:-109.9124)
282- cayenne (4.9333:-52.3333)
283- gasa (27.9037:89.7269)
284- ajdabiya (30.7554:20.2263)
285- bathsheba (13.2167:-59.5167)
286- la palma (8.4028:-78.1453)
287- tuktoyaktuk (69.4541:-133.0374)
gat (LY) Not Found
saleaula (WS) Not Found
290- great 

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame



In [27]:
city_api_df = pd.DataFrame(
                            {"City": cities,
                             "Lat" : lattitudes,
                             "Lng" : longitudes,
                             "Max Temp":max_temps,
                             "Humidity": humidities,
                             "Cloudiness": cloudiness,
                             "Wind Speed": windspeeds,
                             "Country":countries,
                             "Date": dates
                             })


In [28]:

city_api_df.to_csv(output_data_file)
city_api_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,441.000000,441.000000,441.000000,441.000000,441.000000,441.000000,4.410000e+02
mean,21.008304,13.473233,47.424966,75.462585,55.131519,8.922494,1.612135e+09
std,34.523834,89.503156,34.300172,17.293169,38.240993,6.285322,8.572931e+01
min,-54.800000,-179.166700,-45.560000,1.000000,0.000000,0.070000,1.612135e+09
25%,-8.103000,-65.712500,23.000000,68.000000,20.000000,4.140000,1.612135e+09
50%,27.516700,19.961900,61.930000,79.000000,71.000000,7.580000,1.612135e+09
75%,50.400100,86.583300,76.150000,88.000000,90.000000,12.350000,1.612136e+09
max,78.218600,179.316700,91.400000,100.000000,100.000000,53.690000,1.612136e+09


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [31]:
clean_city_data = city_api_df.loc[city_api_df["Humidity"] < 100 ]
clean_city_data.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,433.000000,433.000000,433.000000,433.000000,433.000000,433.000000,4.330000e+02
mean,20.963707,13.500761,47.471686,75.009238,55.006928,8.976351,1.612135e+09
std,34.530524,90.070983,34.549885,17.124136,38.299411,6.309191,8.618016e+01
min,-54.800000,-179.166700,-45.560000,1.000000,0.000000,0.070000,1.612135e+09
25%,-8.103000,-66.115600,23.000000,68.000000,20.000000,4.270000,1.612135e+09
50%,27.432700,19.961900,62.600000,79.000000,69.000000,7.630000,1.612135e+09
75%,50.678900,87.983300,76.240000,87.000000,90.000000,12.370000,1.612136e+09
max,78.218600,179.316700,91.400000,98.000000,100.000000,53.690000,1.612136e+09


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression